In [14]:
import os
import re
import csv
import ast
from typing import Dict, List
from tqdm import tqdm


def parse_csv_file(csv_path: str) -> List[Dict[str, str]]:
    """Parse CSV file into a list of dictionaries."""
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        return list(reader)


def read_file(file_path: str) -> str:
    """Read file content with UTF-8 encoding."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ''


def is_in_string(content: str, pos: int) -> bool:
    """Check if the given position is inside a string."""
    try:
        node = ast.parse(content)
        for n in ast.walk(node):
            # Check for string literals using ast.Constant in Python 3.8 and later
            if isinstance(n, ast.Constant) and isinstance(n.value, str):
                # Check if the position is within the string literal
                if content.find(n.value) <= pos < content.find(n.value) + len(n.value):
                    return True
    except SyntaxError:
        pass
    return False


def process_file(file_path: str, rename_map: Dict[str, str]) -> int:
    """Replace element names with suggested names in the file content, skipping strings and paths."""
    content = read_file(file_path)
    if not content:
        return 0

    original_content = content
    changes_made = 0

    for element_name, suggested_name in rename_map.items():
        pattern = fr'\b{re.escape(element_name)}\b'

        # We will process the content character by character and apply replacements
        i = 0
        while i < len(content):
            # Skip over strings
            if content[i] == '"' or content[i] == "'":
                start = i
                quote_char = content[i]
                i += 1
                while i < len(content) and content[i] != quote_char:
                    i += 1
                i += 1  # Skip the closing quote
                continue

            # Check for the pattern in the non-string area
            match = re.search(pattern, content[i:])
            if match:
                match_start, match_end = match.span()
                match_pos = i + match_start

                # Skip replacement if within a string
                if is_in_string(content, match_pos):
                    i = match_pos + 1  # Continue after the matched part
                    continue

                # Perform the replacement if not in a string
                content = content[:match_pos] + suggested_name + content[match_pos + (match_end - match_start):]
                changes_made += 1
                i = match_pos + len(suggested_name)  # Move index forward to avoid re-processing

            else:
                break

    if content != original_content:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)

    return changes_made


def find_and_replace(repo_path: str, input_csv_file: str, batch_size: int):
    """Process files in batches with progress bars for each input."""
    # Parse the CSV
    try:
        rename_rules = parse_csv_file(input_csv_file)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    if not rename_rules:
        print("No rename rules found in CSV.")
        return

    # Group rename rules by file type
    rules_by_type = {'py': [], 'ts': [], 'svelte': []}
    for rule in rename_rules:
        file_type = rule['File Type'].lstrip('.')  # Strip the dot from the file type (e.g., '.py' -> 'py')
        if file_type in rules_by_type:
            rules_by_type[file_type].append(rule)

    # Collect files to process based on the file types mentioned in the CSV
    allowed_extensions = {'.ts', '.py', '.svelte'}
    files_to_process = []

    for root, _, filenames in os.walk(repo_path):
        # Skip directories that contain 'config' or 'icons' in their name
        if 'config' in root.lower() or 'icons' in root.lower():
            continue
        for filename in filenames:
            file_extension = os.path.splitext(filename)[1].lower()
            file_type = file_extension.lstrip('.')  # Strip the dot from the extension for comparison

            if file_extension in allowed_extensions:
                # Add files based on matching file types
                if file_type == 'py' and rules_by_type['py']:
                    files_to_process.append(os.path.join(root, filename))
                elif file_type == 'ts' and rules_by_type['ts']:
                    files_to_process.append(os.path.join(root, filename))
                elif file_type == 'svelte' and rules_by_type['svelte']:
                    files_to_process.append(os.path.join(root, filename))

    if not files_to_process:
        print("No relevant files found to process.")
        return

    # Process in batches
    total_changes = 0
    for i in range(0, len(rename_rules), batch_size):
        batch = rename_rules[i:i + batch_size]
        rename_map = {rule['Element Name']: rule['Suggested Name'] for rule in batch}

        # Progress bar for the batch
        batch_desc = f"Batch {i // batch_size + 1} ({i + 1}-{min(i + batch_size, len(rename_rules))})"
        with tqdm(total=len(files_to_process), desc=batch_desc, unit="file") as pbar:
            for file_path in files_to_process:
                changes = process_file(file_path, rename_map)
                total_changes += changes
                pbar.update(1)  # Update progress bar for each file processed

        # Prompt to continue after each batch
        if i + batch_size < len(rename_rules):
            continue_processing = input(f"\nProcessed batch {i // batch_size + 1} ({batch_size} rules). Continue? (y/n): ").lower()
            if continue_processing != 'y':
                break

    print(f"\nTotal replacements made: {total_changes}")


if __name__ == "__main__":
    repo_path = r"C:\Users\harold.noble\Desktop\Not workling\ric_new - Copy - Copy\ric"
    input_csv_file = r"C:\Users\harold.noble\Desktop\Not workling\ric_new - Copy - Copy\code_helper\work\processed_output_file.csv"
    batch_size = 15000
    find_and_replace(repo_path, input_csv_file, batch_size)

Batch 1 (1-2740):   0%|          | 0/143 [00:00<?, ?file/s]

Batch 1 (1-2740): 100%|██████████| 143/143 [05:21<00:00,  2.25s/file]


Total replacements made: 44914
